## Creating the dataset for the total loss model in Colombia

In this notebook we create the training dataset

In [2]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
import xgboost as xgb
import boto3
from datetime import datetime

In [2]:
#Read the historico_2019-01-02_2024-01-30.csv which has the claims historical information since 2019

In order to create the training set, we use 2 main datasets: 

1. A file with historical claims. The name of this file changes every day. It has the historical claims
reported since January 2019 up to date. You can find the file here s3://cortex-modeling-andes-local/epiphany-cortex/TABLA_DIARIO/

The one we use in this version is historico_2019-01-02_2024-01-30.csv

Later in this notebook, you will see we keep claims from January 2022 to July 2023.

You can change the dates when training new versions of the model

2. A datatset with the labels. In this version, we use resumentiempos2022 and resumentiempos2023 to get the labels.
You cand find these files here s3://cortex-intl-andes-nppm-latam/totallossco/

For new versions of the model you may need the 2024 file (ResumenTiemposEspecial  122024)
You can download this file here https://libertymutual.sharepoint.com/sites/reportessiniestros/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2Freportessiniestros%2FShared%20Documents%2FFACT%5FTABLES%2FSIPO%2FTIEMPOS&p=true&originalPath=aHR0cHM6Ly9saWJlcnR5bXV0dWFsLnNoYXJlcG9pbnQuY29tLzpmOi9zL3JlcG9ydGVzc2luaWVzdHJvcy9Fc1dOV3ZxWlNHOUtxRTFueXAzd2ZNMEJmX3MwcnRQMzNrTFUwQlZiQl8wRUxRP3J0aW1lPTZiOG1tZ0lwMkVn

If you do not have permissions, please can ask to LOPEZ, WILLIAM

s3://cortex-intl-andes-nppm-latam/totallossco/historico_2019-01-02_2024-01-30.csv

In [7]:
s3 = boto3.client('s3')
s3.download_file('cortex-intl-andes-nppm-latam', 'totallossco/historico_2019-01-02_2024-01-30.csv', 'claims_data.csv')

In [8]:
data = pd.read_csv("claims_data.csv")
data.head(2)

,numero_siniestro,poliza_asociada,placa_vehiculo,cod_fasecolda,descripcion_riesgo,valor_vehiculo,documento,amparo,descripcion_deducible,valor_deducible,...,descripcion_marca,codigo_clase,descripcion_clase,codigo_tipo_vehiculo,descripcion_tipo_vehiculo,anio_modelo,departamento,municipio,tipo_declarante,fecha_proceso
0,708396,213660,HMP692,8006033,HMP692 - RENAULT - DUSTER - EXPRESSIONMT 16,35800000.0,2,760,Deducible Pérdida Parcial Daños,10% con un mínimo de 1 SMMLV,...,RENAULT,6,CAMIONETA PASAJ.,33,DUSTER,2014,Valle del Cauca,PALMIRA,Afiliado,2019-01-02
1,708397,253699,MOU497,9201121,MOU497 - VOLKSWAGEN - JETTA [5] - TRENDLINETP 200,25700000.0,3,760,Deducible Pérdida Parcial Daños,10% con un mínimo de 1 SMMLV,...,VOLKSWAGEN,1,AUTOMOVIL,121,JETTA [5],2010,Antioquia,MEDELLÍN,Afiliado,2019-01-02


In [9]:
data.columns

Index(['numero_siniestro', 'poliza_asociada', 'placa_vehiculo',
       'cod_fasecolda', 'descripcion_riesgo', 'valor_vehiculo', 'documento',
       'amparo', 'descripcion_deducible', 'valor_deducible',
       'descripcion_siniestro', '1.- Sección delantera',
       '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
       '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
       '6.- Sección posterior', '7.- Techo', 'codigo_marca',
       'descripcion_marca', 'codigo_clase', 'descripcion_clase',
       'codigo_tipo_vehiculo', 'descripcion_tipo_vehiculo', 'anio_modelo',
       'departamento', 'municipio', 'tipo_declarante', 'fecha_proceso'],
      dtype='object')

In [10]:
columns_to_keep = ['numero_siniestro', 'valor_vehiculo', 'valor_deducible',
                   'descripcion_siniestro', '1.- Sección delantera',
                   '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
                   '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
                   '6.- Sección posterior', '7.- Techo', 'codigo_marca', 'codigo_clase', 
                   'codigo_tipo_vehiculo', 'anio_modelo',
                   'municipio', 'fecha_proceso'
                  ]

In [11]:
data = data[columns_to_keep]
data.columns

Index(['numero_siniestro', 'valor_vehiculo', 'valor_deducible',
       'descripcion_siniestro', '1.- Sección delantera',
       '2.- Lateral delantero izquierdo', '3.- Lateral delantero derecho',
       '4.- Lateral trasero izquierdo', '5.- Lateral trasero derecho',
       '6.- Sección posterior', '7.- Techo', 'codigo_marca', 'codigo_clase',
       'codigo_tipo_vehiculo', 'anio_modelo', 'municipio', 'fecha_proceso'],
      dtype='object')

In [12]:
data.head(2)

,numero_siniestro,valor_vehiculo,valor_deducible,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,fecha_proceso
0,708396,35800000.0,10% con un mínimo de 1 SMMLV,DETALLE DE OCURRENCIA:El vehículo lo parqueo f...,X,NaN,NaN,X,X,X,X,80,6,33,2014,PALMIRA,2019-01-02
1,708397,25700000.0,10% con un mínimo de 1 SMMLV,Declaración: IBA CONDUCIENDO SALE UN VH QUE ...,X,NaN,NaN,NaN,NaN,NaN,NaN,92,1,121,2010,MEDELLÍN,2019-01-02


In [13]:
data['anio'] = pd.to_datetime(data['fecha_proceso']).dt.year
data['mes'] = pd.to_datetime(data['fecha_proceso']).dt.month

In [14]:
data['anio'].value_counts(dropna=False)

2022    22096
2021    20679
2023    19735
2019    16559
2020    14815
2024     1565
Name: anio, dtype: int64

In this version we are only using 2022 and mid 2023 data for training the model

In [14]:
data = data[(data['anio']>=2022) & (data['anio']<2024)]

In [15]:
data = data.drop(data[(data['anio']==2023) & (data['mes']>=8) & (data['mes']<=12)].index)

In [16]:
data['anio'].value_counts(dropna=False)

2022    22096
2023    11648
Name: anio, dtype: int64

In [17]:
data.reset_index(drop=True, inplace=True)

In [18]:
data = data.drop(columns=["fecha_proceso", "anio", "mes"])
data.head(3)

,numero_siniestro,valor_vehiculo,valor_deducible,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio
0,899767,68900000.0,10% con un mínímo de 6 SMMLV,"El día primero (1) de enero de 2022, siendo ap...",X,X,X,X,X,X,X,16,12,138,2012,GRANADA
1,899768,73700000.0,$950.000,El vehículo venía de Istmina Chocó a recogerm...,X,X,X,X,X,X,X,30,6,134,2020,PEREIRA
2,899770,85990000.0,$950.000,Yo estaba en un semáforo y sentí un impacto en...,NaN,NaN,NaN,X,NaN,X,NaN,42,6,64,2022,"BOGOTÁ, D.C"


In [19]:
data.shape[0]

33744

Now we will create the database with the labels using the files ResumenTiemposEspecial  122022 and ResumenTiemposEspecial  122023 storage in s3://cortex-intl-andes-nppm-latam/totallossco

For new versions of the model you may need the 2024 file (ResumenTiemposEspecial  122024)
You can download this file here https://libertymutual.sharepoint.com/sites/reportessiniestros/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2Freportessiniestros%2FShared%20Documents%2FFACT%5FTABLES%2FSIPO%2FTIEMPOS&p=true&originalPath=aHR0cHM6Ly9saWJlcnR5bXV0dWFsLnNoYXJlcG9pbnQuY29tLzpmOi9zL3JlcG9ydGVzc2luaWVzdHJvcy9Fc1dOV3ZxWlNHOUtxRTFueXAzd2ZNMEJmX3MwcnRQMzNrTFUwQlZiQl8wRUxRP3J0aW1lPTZiOG1tZ0lwMkVn

If you do not have permissions, please can ask to LOPEZ, WILLIAM

In [20]:
s3.download_file('cortex-intl-andes-nppm-latam', 'totallossco/ResumenTiemposEspecial-%20122022.xlsx', 'resumentiempos2022.xlsx')
s3.download_file('cortex-intl-andes-nppm-latam', 'totallossco/ResumenTiemposEspecial-%20122023.xlsx', 'resumentiempos2023.xlsx')

In [21]:
columns_to_keep_resumentiempos = ["Clave", "Amparo", "Siniestro"]
columns_to_rename_resumentiempos = {'Siniestro':'numero_siniestro'}

resumentiempos2022 = pd.read_excel("resumentiempos2022.xlsx")[columns_to_keep_resumentiempos].rename(columns=columns_to_rename_resumentiempos)
resumentiempos2023 = pd.read_excel("resumentiempos2023.xlsx")[columns_to_keep_resumentiempos].rename(columns=columns_to_rename_resumentiempos)

In [22]:
resumentiempos2022.head(2)

,Clave,Amparo,numero_siniestro
0,724478,PPD,899767
1,724479,PTD,899768


In [23]:
resumentiempos = resumentiempos2023.append(resumentiempos2022, ignore_index = True)

/tmp/ipykernel_3950/904119157.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resumentiempos = resumentiempos2023.append(resumentiempos2022, ignore_index = True)


In [24]:
resumentiempos.dtypes

Clave                int64
Amparo              object
numero_siniestro    object
dtype: object

In [25]:
resumentiempos.shape[0]

67251

In [26]:
resumentiempos['numero_siniestro'].isnull().sum()

0

In [27]:
resumentiempos['numero_siniestro'] = resumentiempos['numero_siniestro'].str.extract('(\d+)')
resumentiempos.shape[0]

67251

In [28]:
resumentiempos['numero_siniestro'] = pd.to_numeric(resumentiempos['numero_siniestro'], downcast = 'integer')

In [29]:
resumentiempos.dtypes

Clave                int64
Amparo              object
numero_siniestro     int64
dtype: object

In [30]:
amparos = ["PPD", "PTD"]

In [31]:
resumentiempos = resumentiempos[resumentiempos['Amparo'].isin(amparos)]

In [32]:
resumentiempos['label'] = [1 if x == 'PTD' else 0 for x in resumentiempos['Amparo']]

In [33]:
resumentiempos.dtypes

Clave                int64
Amparo              object
numero_siniestro     int64
label                int64
dtype: object

In [34]:
resumentiempos['label'].value_counts()

0    39286
1     2987
Name: label, dtype: int64

In [35]:
resumentiempos['Amparo'].value_counts()

PPD    39286
PTD     2987
Name: Amparo, dtype: int64

In [36]:
print(f"There are {len(resumentiempos[resumentiempos['label']==1])} total losses in the dataset")

There are 2987 total losses in the dataset


In [37]:
data = data.merge(resumentiempos[["numero_siniestro", "label"]], on = "numero_siniestro", how = "left")

In [38]:
data.shape[0]

33773

In [39]:
data = data[data['label'].notna()]

In [40]:
data.shape[0]

33137

In [41]:
data.drop_duplicates("numero_siniestro", keep=False, inplace=True)
data.shape[0]

33009

In [42]:
data['label'].mean()

0.07016268290466236

In [43]:
data.head(3)

,numero_siniestro,valor_vehiculo,valor_deducible,descripcion_siniestro,1.- Sección delantera,2.- Lateral delantero izquierdo,3.- Lateral delantero derecho,4.- Lateral trasero izquierdo,5.- Lateral trasero derecho,6.- Sección posterior,7.- Techo,codigo_marca,codigo_clase,codigo_tipo_vehiculo,anio_modelo,municipio,label
0,899767,68900000.0,10% con un mínímo de 6 SMMLV,"El día primero (1) de enero de 2022, siendo ap...",X,X,X,X,X,X,X,16,12,138,2012,GRANADA,0.0
1,899768,73700000.0,$950.000,El vehículo venía de Istmina Chocó a recogerm...,X,X,X,X,X,X,X,30,6,134,2020,PEREIRA,1.0
2,899770,85990000.0,$950.000,Yo estaba en un semáforo y sentí un impacto en...,NaN,NaN,NaN,X,NaN,X,NaN,42,6,64,2022,"BOGOTÁ, D.C",0.0


In [44]:
data['label'] = data['label'].astype(int)

In [45]:
data.dtypes

numero_siniestro                     int64
valor_vehiculo                     float64
valor_deducible                     object
descripcion_siniestro               object
1.- Sección delantera               object
2.- Lateral delantero izquierdo     object
3.- Lateral delantero derecho       object
4.- Lateral trasero izquierdo       object
5.- Lateral trasero derecho         object
6.- Sección posterior               object
7.- Techo                           object
codigo_marca                         int64
codigo_clase                         int64
codigo_tipo_vehiculo                 int64
anio_modelo                          int64
municipio                           object
label                                int64
dtype: object

In [46]:
print(f"The % of total losses is {data['label'].mean():.3%}")

The % of total losses is 7.016%


The number is on line with the historical value (6%-8%)

In [47]:
data.to_csv("data_totallossco.csv", index = False)

In [ ]:
#######################